# Calculator

Problem: Given an expression, evaluate it and return the result. Assume that the expression is fully parenthesised and valid. As a stretch goal, try to handle expressions that aren't fully parenthesised (i.e., have implicit brackets).


In [ ]:
def check(a, b):
    assert a == b, f"got {a} but expected {b}"

In [ ]:
import re

operators = {"+", "-", "*", "/"}


def eval(expr):
    stack = []

    i = 0
    while i < len(expr):
        c = expr[i]

        if c == "(":
            stack.append(c)
            i += 1
        elif c in operators:
            stack.append(c)
            i += 1
        elif c.isdigit():
            match = re.search(r"\d+(\.\d+)?", expr[i:])
            if not match:
                raise ValueError("invalid expression")

            stack.append(float(match.group()))
            i += len(match.group())
        elif c == ")":
            rhs = stack.pop()
            op = stack.pop()
            lhs = stack.pop()
            assert stack.pop() == "("  # pop off "("

            if op == "+":
                ans = lhs + rhs
            elif op == "-":
                ans = lhs - rhs
            elif op == "*":
                ans = lhs * rhs
            elif op == "/":
                ans = lhs / rhs
            else:
                raise ValueError("unknown operator " + op)

            stack.append(ans)
            i += 1
        elif c.isspace():
            i += 1
        else:
            raise ValueError("unknown character " + c)

    return stack.pop()

In [ ]:
check(eval("((3 - (2 - 1)) + 4)"), 6.0)
check(eval("((30 - (20 - 10)) + 40)"), 60.0)
check(eval("(3.5 + 2.5)"), 6.0)
check(eval("((10 / 2.5) * 3)"), 12.0)

To improve this, see the implementation in https://cp-algorithms.com/string/expression_parsing.html.


In [ ]:
def eval(expr: str):
    def process_op(op):
        rhs = values.pop()
        lhs = values.pop()

        if op == "+":
            values.append(lhs + rhs)
        elif op == "-":
            values.append(lhs - rhs)
        elif op == "*":
            values.append(lhs * rhs)
        elif op == "/":
            if rhs == 0:
                raise ValueError("division by zero")
            values.append(lhs / rhs)
        elif op == "^":
            values.append(lhs**rhs)
        else:
            raise ValueError("unknown operator " + op)

    def precedence(op):
        if op == "(":
            return 0
        if op in "+-":
            return 1
        elif op in "*/":
            return 2
        elif op == "^":
            return 3
        else:
            raise ValueError("unknown operator " + op)

    def lower_precedence(ch, op):
        if op == "^":  # right associative
            return precedence(ch) < precedence(op)
        return precedence(ch) <= precedence(op)

    values = []
    operators = []

    i = 0
    while i < len(expr):
        ch = expr[i]

        if ch.isspace():
            i += 1
        elif ch.isdigit():
            match = re.search(r"\d+(\.\d+)?", expr[i:])
            if not match:
                raise ValueError("invalid expression")

            values.append(float(match.group()))
            i += len(match.group())
        elif ch == "(":
            operators.append(ch)
            i += 1
        elif ch in "+-*/^":
            while operators and lower_precedence(ch, operators[-1]):
                process_op(operators.pop())
            operators.append(ch)
            i += 1
        elif ch == ")":
            while operators[-1] != "(":
                process_op(operators.pop())
            operators.pop()
            i += 1
        else:
            raise ValueError("unknown character " + ch)

    while operators:
        process_op(operators.pop())

    return values.pop()

In [ ]:
check(eval("1 + 2 + 3"), 6)
check(eval("2 - 3       -     4"), -5)
check(eval("((3 - (2 - 1)) + 4)"), 6.0)
check(eval("((30 - (20 - 10)) + 40)"), 60.0)
check(eval("(3.5 + 2.5)"), 6.0)
check(eval("((10 / 2.5) * 3)"), 12.0)
check(eval("2^3^2"), 512)

We can extend this relatively easily to print out the expression in RPN.


In [ ]:
def rpn(expr: str):
    def process_op(op):
        rhs = values.pop()
        lhs = values.pop()

        if op == "+":
            values.append(lhs + rhs)
        elif op == "-":
            values.append(lhs - rhs)
        elif op == "*":
            values.append(lhs * rhs)
        elif op == "/":
            if rhs == 0:
                raise ValueError("division by zero")
            values.append(lhs / rhs)
        elif op == "^":
            values.append(lhs**rhs)
        else:
            raise ValueError("unknown operator " + op)

        ans.append(op)

    def precedence(op):
        if op == "(":
            return 0
        if op in "+-":
            return 1
        elif op in "*/":
            return 2
        elif op == "^":
            return 3
        else:
            raise ValueError("unknown operator " + op)

    def lower_precedence(ch, op):
        if op == "^":  # right associative
            return precedence(ch) < precedence(op)
        return precedence(ch) <= precedence(op)

    ans = []
    values = []
    operators = []

    i = 0
    while i < len(expr):
        ch = expr[i]

        if ch.isspace():
            i += 1
        elif ch.isdigit():
            match = re.search(r"\d+(\.\d+)?", expr[i:])
            if not match:
                raise ValueError("invalid expression")

            values.append(float(match.group()))
            ans.append(match.group())

            i += len(match.group())
        elif ch == "(":
            operators.append(ch)
            i += 1
        elif ch in "+-*/^":
            while operators and lower_precedence(ch, operators[-1]):
                process_op(operators.pop())
            operators.append(ch)
            i += 1
        elif ch == ")":
            while operators[-1] != "(":
                process_op(operators.pop())
            operators.pop()
            i += 1
        else:
            raise ValueError("unknown character " + ch)

    while operators:
        process_op(operators.pop())

    return " ".join(ans)


In [ ]:
check(rpn("1 + 2 + 3"), "1 2 + 3 +")
check(rpn("((3 - (2 - 1)) + 4)"), "3 2 1 - - 4 +")
check(rpn("((10 / 2.5) * 3)"), "10 2.5 / 3 *")
check(rpn("2^3^2"), "2 3 2 ^ ^")

In [ ]:
# This is pretty hacky and requires restarting the kernel so the history is right
# But it's good enough for now (seeing a basic diff)

from IPython.core.magic import register_line_magic
from difflib import unified_diff


@register_line_magic
def diff_cells(line):
    """
    Compare two executed cells by their execution indices and display the unified diff.
    Usage:
    %diff_cells 5 7
    """
    # Parse the input indices
    indices = line.split()
    if len(indices) != 2:
        raise ValueError(
            "You must provide exactly two execution indices. Example: %diff_cells 5 7"
        )

    try:
        index1, index2 = map(int, indices)
    except ValueError:
        raise ValueError("Indices must be integers. Example: %diff_cells 5 7")

    cell_code_1 = In[index1]
    cell_code_2 = In[index2]

    # Ensure both cells exist
    if cell_code_1 is None:
        raise ValueError(f"Cell with execution index {index1} not found.")
    if cell_code_2 is None:
        raise ValueError(f"Cell with execution index {index2} not found.")

    # Compute the diff
    diff = unified_diff(
        cell_code_1.splitlines(),
        cell_code_2.splitlines(),
        fromfile=f"cell {index1}",
        tofile=f"cell {index2}",
        lineterm="",
    )

    # Print the diff
    print("\n".join(diff))

In [ ]:
%diff_cells 4 6